# 계산 그래프의 이해

계산 그래프를 만든 후 세션을 통해 그래프를 실행한다.
1. 비어있는 계산그래프 생성
2. 계산 그래프에 노드(텐서와 연산)을 추가
3. 그래프 실행
    * 새로운 새션을 시작
    * 그래프에 있는 변수를 초기화
    * 계산 그래프를 실행

In [1]:
import numpy as np
import tensorflow as tf

tf.__version__ # 텐서플로우 버전확인

'2.3.1'

### Variable, 변수형 텐서

변수형 텐서는 텐서의 값이 바뀔 수 있다. `Variable` 클래스로 정의하며 항상 초기값을 지정해 주어야 한다. 자료형과 크기는 초기값으로부터 자동으로 유추한다.

In [2]:
# 실수 변수형 텐서
s = tf.Variable(1.0)

In [3]:
# 벡터 변수형 텐서
v = tf.Variable(tf.ones((2,)))

In [4]:
# 행렬 변수형 텐서
x = tf.Variable(tf.ones((2, 1)))

변수 텐서의 값을 바꿀 때는 `assign`, `assign_add`, `assign_sub` 메서드를 사용한다.

* `assign`: 값을 완전히 할당. `=`에 해당
* `assign_add`: 값을 증가. `+=`에 해당
* `assign_sub`: 값을 감소. `-=`에 해당

In [5]:
# 다음과 같이 하면 안된다!. 변수헝 텐서가 상수형 텐서로 변한다!
# x = tf.ones((2, 1))
x.assign(tf.ones((2, 1)))
x.numpy()

array([[1.],
       [1.]], dtype=float32)

### Eager Execution
TensorFlow v1의 경우 그래프 생성과 그래프 실행을 분리하고, Lazy Evaluation 형태로 세션을 열고 그래프를 실행하는 시점에서 실제값이 계산되는 구조였다. 이는 성능을 위한 선택이었지만 이로 인해 디버깅이 불편하고 직관적인 형태로 프로그래밍이 불가능하다는 점이 계속해서 문제점으로 지적되었다. 따라서 TensorFlow v2에서는 별도의 세션을 통한 실행없이 바로 그래프의 특정값을 계산할수 있는 Eager Execution을 기본적으로 적용하였다. 따라서 모든 코드는 쓰여진 라인 순서에 따라 실행된다.

### Tensorflow function 와 Graph

Tensorflow 1에서 graph는 tensorflow API의 핵심이므로 피할 수가 없었다. 이 때문에 복잡도가 높아졌다. tensorflow 2에서도 graph가 있지만 이전만큼 핵심적이지는 않고 사용하기 쉬워졌다. 


Tensorflow function은 파이토치(PyTorch) 등의 다른 딥러닝 라이브러리에 있는 함수 기능을 본따 텐서플로우 버전 2.0 에서 새로 만들어진 방법이다. 함수를 사용하면 텐서플로우 버전 1에서처럼 플레이스홀더(placeholder)와 계산 그래프 등을 명시적으로 사용하지 않고 선언적으로 계산 과정을 구현할 수 있다.


간단한 예로 세 제곱을 계산하는 함수를 만들어 살펴보자.

In [6]:
# 일반 파이썬 함수 정의
def cube(x):
    return x ** 3

In [7]:
cube(2)

8

In [8]:
# tensor를 사용하여 함수를 호출할 수 있다.
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

### @tf.function 데코레이터

함수에서 수행되는 계산을 분석하고 동일한 작업을 수행하는 계산그래프(computational graph)를 생성한다.

텐서플로는 해당 함수를 최적화하여 계산 그래프를 생성한다. 최적화된 그래프가 준비되면 텐서플로우 함수는 적당한 순서로 그래프 내의 연산을 효율적으로 실행한다. 일반적으로 텐서플로 함수는 원본 파이썬 함수보다 훨씬 빠르게 실행된다. 특히 복잡한 연산 수행에 더 두드러진다. 


사용자 정의 함수를 작성하고 이를 케라스 모형에서 사용할 때는 케라스가 이 함수를 자동으로 텐서플로 함수로 변환한다. 따라서 tf.function을 사용할 필요가 없다.

In [9]:
@tf.function
def tf_cube(x):
    return x ** 3

In [10]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

$ f(x) = Wx + b $ 를 구하는 tensorflow 함수를 구현하면 다음과 같다.

In [11]:
W = tf.Variable(tf.ones(shape=(2,2)), name="W")
b = tf.Variable(tf.zeros(shape=(2)), name="b")
print("W\n", W.numpy())
print("\nb\n", b.numpy())

@tf.function
def forward(x):
    return W * x + b

out_a = forward([1,0])
print("\nout_a\n",out_a)

W
 [[1. 1.]
 [1. 1.]]

b
 [0. 0.]

out_a
 tf.Tensor(
[[1. 0.]
 [1. 0.]], shape=(2, 2), dtype=float32)


## 자동 미분, Automatic Differentiation

텐서플로우에서는 backpropagation에서 gradient 를 계산하기 위해 자동 미분 기능을 제공한다.
TensorFlow에서 자동 미분 기능은 `tf.GradientTape`를 통해 수행된다. 자동 미분 기능은 간단히“autodiff”라고도 한다. 

`tf.GradientTape`는 tensor에 행해지는 각종 operation(연산)을 tracking(추적) 한다. 이러한 tracking을  tensor가 "watched"된다고 표현하기도 한다. 기본적으로 `tf.GradientTape`는 모형에서 훈련되는 변수들(예를 들면 가중치)를 자동으로 "watch"한다. 훈련변수는 `trainable=True` 로 설정하는데 tf.keras 에서 훈련되는 변수들은 `trainable = True` 로 초기화가 이루어 진다. 

일반적인 constant tensor를 수동으로 "watched" 상태로 하기 위해서 명시적으로 watch method를 호출 할 수 있다.


다음과 같은 간단한 예를 보자. :

$$
y = x^2
$$

위 식의 도함수는 다음과 같다.

$$
\frac{d y}{d x} = 2x
$$

 `tf.GradientTape`로 미분을 구하여 보자.

In [12]:
# Set the random seed so things are reproducible
tf.random.set_seed(7)

# Create a random tensor
x = tf.random.normal((2,2))

# Calculate gradient
with tf.GradientTape() as g:
    g.watch(x)
    y = x ** 2
    
dy_dx = g.gradient(y, x)

# Calculate the actual gradient of y = x^2
true_grad = 2 * x

# Print the gradient calculated by tf.GradientTape
print('Gradient calculated by tf.GradientTape:\n', dy_dx)

# Print the actual gradient of y = x^2
print('\nTrue Gradient:\n', true_grad)

# Print the maximum difference between true and calculated gradient
print('\nMaximum Difference:', np.abs(true_grad - dy_dx).max())

Gradient calculated by tf.GradientTape:
 tf.Tensor(
[[1.1966898  0.12552415]
 [0.29263484 0.9696375 ]], shape=(2, 2), dtype=float32)

True Gradient:
 tf.Tensor(
[[1.1966898  0.12552415]
 [0.29263484 0.9696375 ]], shape=(2, 2), dtype=float32)

Maximum Difference: 0.0


### 자동 미분,  Auto diff

변수 텐서 혹은 변수 텐서를 포함하는 연산의 결과로 만들어진 텐서를 입력으로 가지는 함수는 그 변수 텐서로 미분한 값을 계산할 수 있다. 

1. `GradientTape()`로 만들어지는 gradient tape 컨텍스트 내에서 함수값 결과를 저장한 텐서 `y`를 만든다.
2. `tape.gradient(y, x)` 명령으로 변수형 텐서 `x`에 대한 `y`의 미분값을 계산한다.

In [13]:
x = tf.Variable(tf.constant(1.0))

with tf.GradientTape() as tape:
    y = tf.multiply(5, x)

gradient = tape.gradient(y, x) 
gradient.numpy()

5.0

동시에 여러 변수에 대한 그레디언트 벡터를 구할 수도 있다.

In [14]:
x1 = tf.Variable(tf.constant(1.0))
x2 = tf.Variable(tf.constant(1.0))

with tf.GradientTape() as tape:
    y = tf.multiply(x1, x2)

gradients = tape.gradient(y, [x1, x2]) 
gradients[0].numpy(), gradients[1].numpy()

(1.0, 1.0)

이 때 미분하는 텐서가 변수가 아니라 상수형이면 결과로는 `None`이 출력된다.

In [15]:
x = tf.Variable(tf.constant(1.0))
a = tf.constant(1.0)

with tf.GradientTape() as tape:
    y = tf.multiply(a, x)

gradient = tape.gradient(y, a) 
gradient is None

True

만약 상수형 텐서에 대해 미분하고 싶으면 `tape.watch()` 함수를 사용하여 상수형 텐서를 변수형 텐서처럼 바꿔야한다.

In [16]:
with tf.GradientTape() as tape:
    tape.watch(a)
    y = tf.multiply(a, x)

gradient = tape.gradient(y, a) 
gradient.numpy()

1.0